In [1]:
from datetime import datetime
from synthetix_v3.base_perps import TestBasePerps
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import polars as pl

import seaborn as sns
pl.Config.set_fmt_str_lengths(200)

polars.config.Config

In [2]:
# combination of active positions and account liquidations?

In [3]:
base_perps = TestBasePerps()

In [4]:
settled_orders_df = base_perps.get_settled_orders(timestamp=1696260000) # https://goerli.basescan.org/block/10536192 # starting block time Monday, October 2, 2023 5:00:00 PM

In [5]:
polars_df = pl.from_pandas(settled_orders_df)

In [6]:
blacklisted_acct_ids = [
# 1.7014118346046923e+38, # has 1e9 trading vol
                        # 215754283 # has 1e8 trading vol, unsure if whale or an outlier that should be filtered
                        ]

In [7]:
cleaned_settled_orders = (polars_df.select([
    pl.col('orderSettleds_timestamp'),
    pl.col('markets_marketSymbol'),
    pl.col('orderSettleds_accountId'),
    pl.col('orderSettleds_fillPrice'),
    pl.col('orderSettleds_accruedFunding'),
    pl.col('orderSettleds_sizeDelta'),
    pl.col('orderSettleds_newSize'),
    pl.col('orderSettleds_totalFees'),
])
.with_columns([
    pl.from_epoch("orderSettleds_timestamp").alias("datetime"),
    (pl.col('orderSettleds_fillPrice') / 10 ** 18),
    (pl.col('orderSettleds_sizeDelta') / 10 ** 18),
    (pl.col('orderSettleds_accruedFunding') / 10 ** 18),
    (pl.col('orderSettleds_newSize') / 10 ** 18),
    (pl.col('orderSettleds_totalFees') / 10 ** 18),
])
.with_columns([
    (pl.col('orderSettleds_sizeDelta').abs() * pl.col('orderSettleds_fillPrice')).alias('size_usd')
])
.filter(~pl.col('orderSettleds_accountId').is_in(blacklisted_acct_ids))
)

In [8]:
cleaned_settled_orders.head(5)

orderSettleds_timestamp,markets_marketSymbol,orderSettleds_accountId,orderSettleds_fillPrice,orderSettleds_accruedFunding,orderSettleds_sizeDelta,orderSettleds_newSize,orderSettleds_totalFees,datetime,size_usd
i64,str,f64,f64,f64,f64,f64,f64,datetime[μs],f64
1697020928,"""ETH""",1.1224e9,1577.801896,-44.243833,86.997996,86.997996,68.632802,2023-10-11 10:42:08,137265.603517
1697023666,"""ETH""",1.1224e9,1572.851923,-0.229878,1.0,87.997996,0.31457,2023-10-11 11:27:46,1572.851923
1697023732,"""ETH""",1.1224e9,1572.968366,-0.005573,10.0,97.997996,3.145937,2023-10-11 11:28:52,15729.68366
1697023834,"""ETH""",1.1224e9,1573.181377,-0.008599,-97.997996,0.0,77.084311,2023-10-11 11:30:34,154168.622772
1697023922,"""ETH""",1.1224e9,1572.327512,-0.007403,3.283109,3.283109,1.032424,2023-10-11 11:32:02,5162.12198


### Liquidations

In [9]:
market_price_df = base_perps.get_market_updates()

In [10]:
pos_liqs_df = base_perps.get_position_liquidations()

In [11]:
position_liquidations_df = pl.from_pandas(pos_liqs_df)

In [12]:
# normalize numbers
market_price_polars_df = pl.from_pandas(market_price_df).with_columns([
    pl.from_epoch('marketUpdateds_timestamp').dt.round("1s").alias('datetime'),
    pl.col('marketUpdateds_price') / 10**18,
    pl.col('marketUpdateds_skew') / 10**18,
    pl.col('marketUpdateds_size') / 10**18,
    pl.col('marketUpdateds_sizeDelta') / 10**18,
    pl.col('marketUpdateds_currentFundingRate') / 10**18,
    pl.col('marketUpdateds_currentFundingVelocity') / 10**18
])

# normalize numbers
position_liquidations_df = position_liquidations_df.with_columns([
    pl.from_epoch('positionLiquidateds_timestamp').dt.round("1s").alias('datetime'),
    pl.col('positionLiquidateds_amountLiquidated') / 10**18,
    pl.col('positionLiquidateds_currentPositionSize') / 10**18,
    ]).filter(pl.col("datetime") > datetime(2023, 10, 2))

In [13]:
position_liquidations_usd_df = position_liquidations_df.join(market_price_polars_df, on='datetime', how='left')

In [14]:
position_liquidations_usd_df.head(5)

positionLiquidateds_id,positionLiquidateds_timestamp,positionLiquidateds_accountId,markets_id,positionLiquidateds_amountLiquidated,positionLiquidateds_currentPositionSize,markets_marketSymbol,datetime,marketUpdateds_timestamp,markets_id_right,marketUpdateds_price,marketUpdateds_skew,marketUpdateds_size,marketUpdateds_sizeDelta,marketUpdateds_currentFundingRate,marketUpdateds_currentFundingVelocity,markets_marketSymbol_right
str,i64,f64,i64,f64,f64,str,datetime[μs],i64,i64,f64,f64,f64,f64,f64,f64,str
"""100-170141183460469231731687303715884105794-11302443""",1697798502,1.7014e38,100,109.3068,0.0,"""ETH""",2023-10-20 10:41:42,1697798502,100,1624.86338,763.810779,6656.744796,109.3068,0.000473,0.006874,"""ETH"""
"""100-170141183460469231731687303715884105794-11302435""",1697798486,1.7014e38,100,699.993,-109.3068,"""ETH""",2023-10-20 10:41:26,1697798486,100,1624.86338,654.503979,6766.051596,699.993,0.000472,0.005891,"""ETH"""
"""100-170141183460469231731687303715884105794-11302423""",1697798462,1.7014e38,100,699.993,-809.2998,"""ETH""",2023-10-20 10:41:02,1697798462,100,1629.715159,-45.489021,7466.044596,699.993,0.000472,-0.000409,"""ETH"""
"""200-170141183460469231731687303715884105794-11302414""",1697798444,1.7014e38,200,145.6077,0.0,"""BTC""",2023-10-20 10:40:44,1697798444,200,30230.824183,68.673255,660.845055,145.6077,0.003956,0.000618,"""BTC"""
"""200-170141183460469231731687303715884105794-11302414""",1697798444,1.7014e38,200,145.6077,0.0,"""BTC""",2023-10-20 10:40:44,1697798444,100,1629.715159,-745.482021,8166.037596,699.993,0.000473,-0.006709,"""ETH"""


In [15]:
position_liquidations_usd_df.columns

['positionLiquidateds_id',
 'positionLiquidateds_timestamp',
 'positionLiquidateds_accountId',
 'markets_id',
 'positionLiquidateds_amountLiquidated',
 'positionLiquidateds_currentPositionSize',
 'markets_marketSymbol',
 'datetime',
 'marketUpdateds_timestamp',
 'markets_id_right',
 'marketUpdateds_price',
 'marketUpdateds_skew',
 'marketUpdateds_size',
 'marketUpdateds_sizeDelta',
 'marketUpdateds_currentFundingRate',
 'marketUpdateds_currentFundingVelocity',
 'markets_marketSymbol_right']

In [16]:
cleaned_settled_orders.columns

['orderSettleds_timestamp',
 'markets_marketSymbol',
 'orderSettleds_accountId',
 'orderSettleds_fillPrice',
 'orderSettleds_accruedFunding',
 'orderSettleds_sizeDelta',
 'orderSettleds_newSize',
 'orderSettleds_totalFees',
 'datetime',
 'size_usd']

In [17]:
# rename columns for concatting
position_liq_final = position_liquidations_usd_df.rename({
    'positionLiquidateds_accountId': 'accountId', 
    'markets_marketSymbol_right': 'marketSymbol',
    'positionLiquidateds_amountLiquidated': 'size_delta',
    'positionLiquidateds_currentPositionSize': 'size',
    'marketUpdateds_price': 'price'
     }).with_columns([
    pl.lit(True).alias('liquidation')
]).select('datetime', 'accountId', 'marketSymbol', 'size_delta', 'size', 'price', 'liquidation')

settled_order_final = cleaned_settled_orders.rename({
    'orderSettleds_accountId': 'accountId', 
    'markets_marketSymbol': 'marketSymbol',
    'orderSettleds_sizeDelta': 'size_delta',
    'orderSettleds_newSize': 'size',
    'orderSettleds_fillPrice': 'price',
     }).with_columns([
    pl.lit(False).alias('liquidation')]).select('datetime', 'accountId', 'marketSymbol', 'size_delta', 'size', 'price', 'liquidation')

In [18]:
# get the overlap between unique accuonts
unique_liq_accts = position_liq_final['accountId'].unique().to_list()
unique_orders_accts = settled_order_final['accountId'].unique().to_list()

print(len(set(unique_liq_accts)))
print(len(set(unique_orders_accts)))

27
49


In [19]:
# are there liquidation accounts that are not in unique order accts?
print(len(set(unique_liq_accts) - set(unique_orders_accts)))


3


### Concat data and analyze results

In [20]:
concat_df = pl.concat([position_liq_final, settled_order_final])

In [21]:
concat_df.filter(pl.col('liquidation') == True).shape

(385, 7)

In [22]:
concat_df.filter(pl.col('liquidation') == False).shape

(2168, 7)

In [23]:
filtered_acct = concat_df['accountId'][0]

In [24]:
expanded_number = f'{filtered_acct:.0f}'
print(expanded_number)

170141183460469231731687303715884105728


In [32]:
settled_order_final.filter(pl.col('accountId') == filtered_acct).filter(pl.col('marketSymbol') == 'ETH')

datetime,accountId,marketSymbol,size_delta,size,price,liquidation
datetime[μs],f64,str,f64,f64,f64,bool
2023-10-02 18:05:04,1.7014e38,"""ETH""",0.7604,1.6502,1689.579753,false
2023-10-02 20:02:46,1.7014e38,"""ETH""",-1.6502,0.0,1670.567628,false
2023-10-02 20:03:26,1.7014e38,"""ETH""",0.2664,0.2664,1670.044143,false
2023-10-06 09:55:18,1.7014e38,"""ETH""",0.7489,1.0153,1632.600741,false
2023-10-06 10:00:34,1.7014e38,"""ETH""",-1.0153,0.0,1627.12249,false
2023-10-06 10:02:48,1.7014e38,"""ETH""",0.583,0.583,1626.74543,false
2023-10-06 10:04:54,1.7014e38,"""ETH""",-0.583,0.0,1627.253994,false
2023-10-06 10:05:42,1.7014e38,"""ETH""",0.7765,0.7765,1626.582965,false
2023-10-02 17:22:50,1.7014e38,"""ETH""",95.773,95.773,1685.827036,false


In [29]:
concat_df.filter(pl.col('accountId') == filtered_acct).filter(pl.col('marketSymbol') == 'ETH').sort(by='datetime', descending=True).head(20)

datetime,accountId,marketSymbol,size_delta,size,price,liquidation
datetime[μs],f64,str,f64,f64,f64,bool
2023-10-20 19:22:04,1.7014e38,"""ETH""",1116.7678,1116.7678,1607.532675,false
2023-10-20 16:05:34,1.7014e38,"""ETH""",622.3908,622.3908,1605.855257,false
2023-10-20 12:13:46,1.7014e38,"""ETH""",618.366,618.366,1613.527087,false
2023-10-20 11:28:04,1.7014e38,"""ETH""",-2059.0,-2061.058,1617.960681,false
2023-10-20 11:27:08,1.7014e38,"""ETH""",-2.058,-2.058,1620.019083,false
2023-10-20 10:54:04,1.7014e38,"""ETH""",-308.9051,0.0,1619.331493,false
2023-10-20 10:41:42,1.7014e38,"""ETH""",109.3068,0.0,1624.86338,true
2023-10-20 10:41:26,1.7014e38,"""ETH""",699.993,-109.3068,1624.86338,true
2023-10-20 10:41:02,1.7014e38,"""ETH""",699.993,-809.2998,1629.715159,true
